# 项目导图
你在这个项目中的任务如下：

### 清洗数据包括：
- 收集数据
- 评估数据
- 清洗数据
### 对清洗过的数据进行储存、分析和可视化
### 汇报 1) 你的数据清洗过程 和 2) 你的数据分析和可视化

项目提交
在这一项目中，你将收集、评估和清理数据，然后通过分析、可视化和/或建模来处理数据。

在你提交前:

- 1 确保你符合审阅标准中所有条目的规范要求。只有符合所有规范要求，你的项目才能“符合审阅标准”。
- 2 确保你在你的项目文件中没有包含你的API密钥和访问令牌。
- 3 如果你在项目工作区中完成项目，请确保工作区中包含以下文件，然后单击项目工作区页面右下角的“提交项目”：

    - wrangle_act.ipynb：用于收集、评估、清理、分析和可视化数据代码
    - wrangle_report.pdf：数据整理步骤的文档：收集，评估和清理
    - act_report.pdf：观察并分析最终数据的文档
    - twitter_archive_enhanced.csv：给定的文件
    - ==image_predictions.tsv：以编程方式下载的文件
    - tweet_json.txt：通过API构建的文件
    - twitter_archive_master.csv：合并和清理数据
    - 任何附加文件（例如，用于存储干净数据的附加文件或数据库文件）
- 4 如果你并没有在优达学城教室里在线完成了项目，而是在教室外完成，请将上面列出的文件打包成zip 压缩文件或从GitHub仓库中推出，然后单击此页面上的“提交项目”按钮。

## 收集数据
- 从github仓库[https://github.com/udacity/new-dand-advanced-china](https://github.com/udacity/new-dand-advanced-china)中下载文件

In [7]:
# -*- coding=utf-8 -*-
import requests

def download(url):
    with open(url.split('/')[-1], mode="wb") as f:
        respone = requests.get(url)
        f.write(respone.content)
        
# 下载 image-predictions.tsv
image_prediction_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv"
download(image_prediction_url)
# 下载 twitter_archive_enhanced.csv
twitter_archive_enhanced_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/twitter-archive-enhanced.csv"
download(twitter_archive_enhanced_url)
# 下载 tweet_json.txt
tweet_json_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/tweet_json.txt"
download(tweet_json_url)

In [4]:
# 使用pandas读取数据文件
import pandas as pd

In [2]:
image_predictions = pd.read_csv("image-predictions.tsv", sep='\t')
image_predictions.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [3]:
witter_archive_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
witter_archive_enhanced.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [2]:
# 处理tweet_json.txt
import json
with open("tweet_json.txt", 'r') as json_file:
    for data in json_file.readlines():
        print(json.dumps(json.loads(data), indent=4, sort_keys=False, ensure_ascii=True))
        break

{
    "contributors": null, 
    "truncated": false, 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "id": 892420643555336193, 
    "favorite_count": 39492, 
    "full_text": "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "retweeted": false, 
    "coordinates": null, 
    "entities": {
        "symbols": [], 
        "user_mentions": [], 
        "hashtags": [], 
        "urls": [], 
        "media": [
            {
                "expanded_url": "https://twitter.com/dog_rates/status/892420643555336193/photo/1", 
                "display_url": "pic.twitter.com/MgUWQ76dJU", 
                "url": "https://t.co/MgUWQ76dJU", 
                "media_url_https": "https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg", 
                "id_str": "892420639486877696", 
                "sizes": {
    

In [26]:
# 逐行读取tweet_json.txt文件并添加到pandas DataFrame中，（至少）包含 tweet ID、retweet_count和favorite_count字段...
# 提出问题，为什么不是放到list中，然后通过 'id_str': list1, 'fav_count': list2的方式创建df？
# 我的思考：如果数据量很大，那么可能会导致生成的list的过大，而处理一条则append一条的方式，不要求较大的运行内存。
df_tweet = pd.DataFrame()

with open("tweet_json.txt", 'r') as json_file:
    for data in json_file.readlines():
        dict_data = json.loads(data)
        s_tweet = pd.Series([dict_data[u'id'], dict_data[u'retweet_count'], dict_data[u'favorite_count']], index=['id', 'retweet_count', 'favorite_count'])
        retweet_count = dict_data[u'retweet_count']
        df_tweet = df_tweet.append(s_tweet, ignore_index=True)

df_tweet.head(3)

,favorite_count,id,retweet_count
0,39492.0,892420643555336193,8842.0
1,33786.0,892177421306343426,6480.0
2,25445.0,891815181378084864,4301.0


In [25]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
favorite_count    2352 non-null float64
id_str            2352 non-null object
retweet_count     2352 non-null float64
dtypes: float64(2), object(1)
memory usage: 55.2+ KB


### 要什么数据？
项目推荐数据
- id
- retweet_count
- favorite_count

额外我想研究的数据
- full_text
- retweeted
- source
- favorited

## 评估数据

## 清洗数据

In [ ]:
# 对清洗过的数据进行储存、分析和可视化